# Recurrent Neural Network

## Part 1 - Data Preprocessing

### Importing the libraries

In [1]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


### Importing the data set and splitting it into train & test set

In [2]:
# dataset = pd.read_csv('bitcoin_price_Training.csv')
# training_set = dataset.iloc[:, 1:2].values
# test_set = training_set[1:300]
# training_set = training_set[300:len(training_set)]
# load dataset
series = read_csv('bitcoin_price_Training.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
series_2 = read_csv('bitcoin2.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
series = series.iloc[:, 0:1]
series = series.iloc[::-1]
series_2 = series_2.iloc[::-1]
# series.index = series.index.to_period('M')
print(series_2)

Date
2017-08-07   NaN
2017-08-08   NaN
2017-08-09   NaN
2017-08-10   NaN
2017-08-11   NaN
              ..
2018-08-02   NaN
2018-08-03   NaN
2018-08-04   NaN
2018-08-05   NaN
2018-08-06   NaN
Name: Open, Length: 365, dtype: float64


### Feature Scaling

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(training_set)

### Split into train and test sets

In [3]:
# split into train and test sets

index = []

for i in series.index:
    index.append(i)

X = series.values
size = int(len(X) * 0.8)
train, test = X[0:size], X[size:len(X)]
train_i, test_i = index[0:size], index[size:len(index)]

test_2_i = []

for i in series_2.index:
    test_2_i.append(i)

print(test_2_i)




[Timestamp('2017-08-07 00:00:00'), Timestamp('2017-08-08 00:00:00'), Timestamp('2017-08-09 00:00:00'), Timestamp('2017-08-10 00:00:00'), Timestamp('2017-08-11 00:00:00'), Timestamp('2017-08-12 00:00:00'), Timestamp('2017-08-13 00:00:00'), Timestamp('2017-08-14 00:00:00'), Timestamp('2017-08-15 00:00:00'), Timestamp('2017-08-16 00:00:00'), Timestamp('2017-08-17 00:00:00'), Timestamp('2017-08-18 00:00:00'), Timestamp('2017-08-19 00:00:00'), Timestamp('2017-08-20 00:00:00'), Timestamp('2017-08-21 00:00:00'), Timestamp('2017-08-22 00:00:00'), Timestamp('2017-08-23 00:00:00'), Timestamp('2017-08-24 00:00:00'), Timestamp('2017-08-25 00:00:00'), Timestamp('2017-08-26 00:00:00'), Timestamp('2017-08-27 00:00:00'), Timestamp('2017-08-28 00:00:00'), Timestamp('2017-08-29 00:00:00'), Timestamp('2017-08-30 00:00:00'), Timestamp('2017-08-31 00:00:00'), Timestamp('2017-09-01 00:00:00'), Timestamp('2017-09-02 00:00:00'), Timestamp('2017-09-03 00:00:00'), Timestamp('2017-09-04 00:00:00'), Timestamp('20

## Walk-forward validation

In [4]:
# walk-forward validation
history = [x for x in train]
original_test_l = len(test)

predictions = list()
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,1))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	# print('predicted=%f, expected=%f' % (yhat, obs))

p2 = list()
for t in range(len(test_2_i)):
	model = ARIMA(predictions, order=(5,1,1))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	p2.append(yhat)

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


## Part 3 - Evaluate forecasts and visualising the results

### Evaluate forecasts

In [5]:
# evaluate forecasts
# rmse = sqrt(mean_squared_error(test, predictions))
# print('Test RMSE: %.3f' % rmse)
print(len(predictions))

X_all = test_i
for i in series_2.index:
    X_all.append(i)




678


### Visualising the results

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# fig = px.line(predictions, title="Bitcoin prices") 
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=test_i,
    y=np.reshape(test,len(test)),
    name = 'Test', 
    connectgaps=True # override default to connect the gaps
))
fig.add_trace(go.Scatter(
    x=X_all,
    y=np.reshape(predictions,len(predictions)),
    name = 'Predition', 
    connectgaps=True # override default to connect the gaps
))
fig.show()

## Part 4 -  Auto-ARIMA!

### Installing ‘pmdarima’ package

In [7]:
! pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Test for Stationarity

#### Method #1

In [8]:
from pmdarima.arima import ADFTest
adf_test = ADFTest(alpha = 0.05)
adf_test.should_diff(series)
# Not sure if it's correct!

(0.48449266173024474, True)

#### Method #2

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(series['Open'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
  print('\t%s: %.3f' % (key, value))

# Is p-value < 0.05 ?

ADF Statistic: -2.801173
p-value: 0.058106
Critical Values:
	1%: -3.435
	5%: -2.863
	10%: -2.568


#### Finding the D, in case we need it

##### Plot the original, 1st and 2nd order Diff of the series

In [ ]:
pyplot.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
 
# Original Series
fig, (ax1, ax2, ax3) = pyplot.subplots(3)
ax1.plot(series.Open); ax1.set_title('Original Series'); ax1.axes.xaxis.set_visible(False)
# 1st Differencing
ax2.plot(series.Open.diff()); ax2.set_title('1st Order Differencing'); ax2.axes.xaxis.set_visible(False)
# 2nd Differencing
ax3.plot(series.Open.diff().diff()); ax3.set_title('2nd Order Differencing')
pyplot.show()

##### Autocorrelation

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
fig, (ax1, ax2, ax3) = pyplot.subplots(3)
plot_acf(series.Open, ax=ax1)
plot_acf(series.Open.diff().dropna(), ax=ax2)
plot_acf(series.Open.diff().diff().dropna(), ax=ax3)

### Train and Test split

In [10]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

size = int(len(series) * 0.8)
train = series[:size]
test = series[size - len(series):]

# fig = px.line(predictions, title="Bitcoin prices") 
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = train.index,
    y = train['Open'],
    name = 'Train', 
    connectgaps=True # override default to connect the gaps
))
fig.add_trace(go.Scatter(
    x = test.index,
    y = test['Open'],
    name = 'Test', 
    connectgaps=True # override default to connect the gaps
))
fig.add_trace(go.Scatter(
    x = series_2.index,
    y = p2,
    name = "+1 Year prediction",
    connectgaps=True # override default to connect the gaps
))
fig.show()

In [ ]:
from pmdarima.arima import auto_arima

model = auto_arima(train, start_p=2, d=1, start_q=1,
                  max_p=5, max_d=1, max_q=1,
                      m=5,          
                      seasonal=False,   
                      error_action='warn',
                      trace=True,
                      suppress_warnings=True, 
                      stepwise=True,
                   random_state=0, n_fits=20)

# model = auto_arima(train, start_p=1, start_q=1,
#                       test='adf',
#                       max_p=5, max_q=5,
#                       m=1,             
#                       d=1,          
#                       seasonal=False,   
#                       start_P=0, 
#                       D=None, 
#                       trace=True,
#                       error_action='ignore',  
#                       suppress_warnings=True, 
#                       stepwise=True)

/usr/local/lib/python3.7/dist-packages/pmdarima/arima/_validation.py:62: UserWarning:

m (5) set for non-seasonal fit. Setting to 0



Performing stepwise search to minimize aic
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=11406.941, Time=1.05 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=11415.822, Time=0.21 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=11417.768, Time=0.32 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=11417.756, Time=0.30 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=11414.149, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=11419.270, Time=0.82 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=11409.624, Time=0.19 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=11394.431, Time=1.18 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=11395.216, Time=0.54 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=11375.491, Time=1.80 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=11390.214, Time=0.66 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=11373.183, Time=1.89 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=11378.251, Time=0.94 sec
 ARIMA(5,1,1)(0,0,0)[0]             : AIC=11371.449, Time=0.77 sec
 ARIMA(4,1,1)(0,0,0

In [ ]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1249
Model:               SARIMAX(5, 1, 1)   Log Likelihood               -5678.724
Date:                Thu, 01 Dec 2022   AIC                          11371.449
Time:                        23:35:13   BIC                          11407.354
Sample:                    04-28-2013   HQIC                         11384.948
                         - 09-27-2016                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4261      0.079      5.366      0.000       0.270       0.582
ar.L2         -0.0713      0.013     -5.593      0.000      -0.096      -0.046
ar.L3         -0.0688      0.012     -5.654      0.000      -0.093      -0.045
ar.L4          0.1232      0.018      6.885      0.000       0.088       0.158
ar.L5          0.0846      0.014      5.974      0.000       0.057       0.112
ma.L1         -0.4364      0.079     -5.543      0.000      -0.591      -0.282
sigma2       524.5255      6.072     86.380      0.000     512.624     536.427
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):             34547.59
Prob(Q):                              0.93   Prob(JB):                         0.00
Heteroskedasticity (H):               0.15   Skew:                            -0.88
Prob(H) (two-sided):                  0.00   Kurtosis:                        28.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
pred = pd.DataFrame(model.predict(X=test.index, n_periods=len(series)-size), index=test.index)
pred.columns = ['Predicted price']
print(pred)

            Predicted price
Date                       
2016-09-28       608.068585
2016-09-29       608.254619
2016-09-30       607.577693
2016-10-01       607.996827
2016-10-02       608.826569
...                     ...
2017-08-02       609.590519
2017-08-03       609.590519
2017-08-04       609.590519
2017-08-05       609.590519
2017-08-06       609.590519

[313 rows x 1 columns]


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = test.index,
    y = test['Open'],
    name = 'Test', 
    connectgaps=True # override default to connect the gaps
))
fig.add_trace(go.Scatter(
    x=pred.index,
    y=pred['Predicted price'],
    name = 'Predition', 
    connectgaps=True # override default to connect the gaps
))
fig.show()